The data is downloaded by the following solar panel conditions:

### Número de paneles necesarios para abastecer una casa

El número de paneles necesarios para abastecer una casa depende de varios factores, incluyendo el consumo promedio de energía de la casa, la irradiación solar en la ubicación, la eficiencia del sistema y las horas de sol al día. Vamos a desglosarlo:

---

### 1. **Consumo promedio de energía de una casa**
El consumo de energía varía según la región y los hábitos de los habitantes. Por ejemplo:

- En **Colombia**, una casa promedio consume entre **250 y 300 kWh/mes**.
- Esto equivale a aproximadamente:
  \[
  \text{Consumo diario promedio} = \frac{300 \, \text{kWh}}{30 \, \text{días}} = 10 \, \text{kWh/día}.
  \]

---

### 2. **Producción diaria por panel solar**
Para un panel de **0.59 kWp**, la energía diaria que genera depende de las horas de sol pico (HSP), que son las horas equivalentes de irradiación solar de 1000 W/m² al día. 

#### Supongamos:
- **HSP en Colombia**: Entre **4 y 6 horas/día** (depende de la región).
- Energía diaria por panel:
  \[
  \text{Energía generada} = \text{Potencia pico (kWp)} \times \text{HSP}.
  \]
  Por ejemplo, si HSP = 5:
  \[
  \text{Energía generada} = 0.59 \, \text{kWp} \times 5 \, \text{horas} = 2.95 \, \text{kWh/día}.
  \]

---

### 3. **Número de paneles necesarios**
Para cubrir **10 kWh/día**, se calcula:
\[
\text{Número de paneles} = \frac{\text{Consumo diario}}{\text{Energía diaria por panel}}.
\]
\[
\text{Número de paneles} = \frac{10}{2.95} \approx 4 \, \text{paneles}.
\]

---

### 4. **Factores adicionales**
- **Pérdidas del sistema**: Por cableado, inversor, inclinación, etc., se pierde entre **10% y 20%** de la energía.
  - Ajustando por pérdidas (\(20\%\)):
    \[
    \text{Número ajustado de paneles} = 4 \times 1.2 \approx 5 \, \text{paneles}.
    \]
- **Autonomía o almacenamiento**: Si deseas almacenar energía (por ejemplo, con baterías) para uso nocturno o en días nublados, necesitarás más paneles y un sistema de almacenamiento adecuado.

---

### **Conclusión**
Para una casa promedio en Colombia con un consumo de **10 kWh/día**, necesitarías alrededor de **5 paneles solares** de **0.59 kWp** cada uno, asumiendo condiciones óptimas de irradiación y un sistema bien diseñado.



Por tanto, para descargar los datos, consideramos valores tomados cada hora desde el año 2018 hasta el año 2023 para los 32 departamentos de Colombia.

Con una pérdida del sistema de 18% y un kWp de 0.59 para 5 paneles solares de las carácterísticas del nuestro particular.

In [77]:
# Import libraries

import numpy as np
import pandas as pd
import os

In [78]:
# Load the data

# Ruta de la carpeta con los archivos CSV
folder_path = "../data/AllDepartments/"

# Lista para almacenar los DataFrames
dataframes = []

# Recorrer cada archivo CSV en la carpeta
for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):  # Verificar que sea un archivo CSV
        # Ruta completa del archivo
        file_path = os.path.join(folder_path, file_name)
        
        # Leer el archivo, ignorando las primeras N filas y últimas M
        df = pd.read_csv(
            file_path,
            skiprows=10,       
            skipfooter=11,     
            engine="python"   # Necesario para skipfooter
        )
        
        # Añadir el nombre del departamento como columna
        department_name = file_name.replace(".csv", "")
        df["Departamento"] = department_name
        
        # Limpiar columnas innecesarias 
        df = df[["time", "P", "G(i)", "H_sun","T2m","WS10m","Departamento"]]
        
        # Agregar el DataFrame limpio a la lista
        dataframes.append(df)

# Combinar todos los DataFrames en uno solo
combined_df = pd.concat(dataframes, ignore_index=True)

In [79]:
combined_df.sample(10)

,time,P,G(i),H_sun,T2m,WS10m,Departamento
1266071,20180618:2330,0.00,0.00,0.00,22.37,1.17,vaupes
1330675,20191101:1930,273.32,642.81,39.84,29.29,1.31,Arauca
1296672,20211215:0030,0.00,0.00,0.00,24.70,1.17,vaupes
1388665,20200614:0130,0.00,0.00,0.00,27.54,1.66,Sucre
1580123,20180419:1130,25.28,70.36,10.82,22.21,0.69,Amazonas
1192121,20220109:1730,421.97,1037.22,64.67,25.40,0.97,Huila
142038,20220317:0630,0.00,0.00,0.00,26.01,1.66,Magdalena
1045730,20230428:0230,0.00,0.00,0.00,27.61,4.62,SanAndres
1252068,20221112:1230,160.66,352.29,22.31,15.37,0.90,Risaralda
874970,20211102:0230,0.00,0.00,0.00,10.19,0.07,Caldas


In [80]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682688 entries, 0 to 1682687
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   time          1682688 non-null  object 
 1   P             1682688 non-null  float64
 2   G(i)          1682688 non-null  float64
 3   H_sun         1682688 non-null  float64
 4   T2m           1682688 non-null  float64
 5   WS10m         1682688 non-null  float64
 6   Departamento  1682688 non-null  object 
dtypes: float64(5), object(2)
memory usage: 89.9+ MB


In [81]:
data = combined_df.copy()

In [82]:
# Correct the time column

# Convert to datetime
data['time'] = pd.to_datetime(data['time'], format='%Y%m%d:%H%M')
data.sample(10)

,time,P,G(i),H_sun,T2m,WS10m,Departamento
876324,2021-12-28 12:30:00,15.88,47.63,17.62,9.24,0.83,Caldas
631260,2018-01-11 12:30:00,193.28,433.87,19.51,21.89,0.69,Caqueta
1351518,2022-03-19 06:30:00,0.00,0.00,0.00,25.41,2.34,Arauca
195755,2022-05-03 11:30:00,5.81,24.59,6.63,19.40,0.97,Cauca
651498,2020-05-03 18:30:00,83.80,202.32,61.94,25.75,0.62,Caqueta
1358517,2023-01-04 21:30:00,246.19,569.81,15.05,30.84,2.83,Arauca
1526186,2018-02-22 02:30:00,0.00,0.00,0.00,26.40,2.21,Guaviare
1321207,2018-10-03 07:30:00,0.00,0.00,0.00,24.08,1.52,Arauca
1494846,2020-07-26 06:30:00,0.00,0.00,0.00,22.49,0.28,Guainia
38523,2022-05-25 03:30:00,0.00,0.00,0.00,25.11,1.24,Cordoba


In [83]:
# add loingitude and latitude

departments_coordinates = {
    'Amazonas': {'latitude': -3.5, 'longitude': -70.2},
    'Antioquia': {'latitude': 6.5, 'longitude': -75.5},
    'Arauca': {'latitude': 7.1, 'longitude': -70.7},
    'Atlantico': {'latitude': 10.6, 'longitude': -74.2},
    'Bolivar': {'latitude': 10.3, 'longitude': -75.5},
    'Boyaca': {'latitude': 5.5, 'longitude': -73.4},
    'Caldas': {'latitude': 5.1, 'longitude': -75.5},
    'Caqueta': {'latitude': 1.6, 'longitude': -75.6},
    'Casanare': {'latitude': 5.9, 'longitude': -72.4},
    'Cauca': {'latitude': 2.5, 'longitude': -76.6},
    'Cesar': {'latitude': 10.4, 'longitude': -73.8},
    'Choco': {'latitude': 5.7, 'longitude': -77.6},
    'Cordoba': {'latitude': 8.6, 'longitude': -75.9},
    'Cundinamarca': {'latitude': 4.1, 'longitude': -74.2},
    'Guaviare': {'latitude': 3.2, 'longitude': -72.6},
    'Guainia': {'latitude': 3.9, 'longitude': -67.5},
    'Huila': {'latitude': 2.9, 'longitude': -75.3},
    'Guajira': {'latitude': 11.0, 'longitude': -71.9},
    'Magdalena': {'latitude': 9.9, 'longitude': -74.2},
    'Meta': {'latitude': 3.6, 'longitude': -73.3},
    'Nariño': {'latitude': 1.2, 'longitude': -77.0},
    'NorteSantander': {'latitude': 7.8, 'longitude': -72.9},
    'Putumayo': {'latitude': 1.0, 'longitude': -75.4},
    'Quindio': {'latitude': 4.5, 'longitude': -75.6},
    'Risaralda': {'latitude': 5.0, 'longitude': -75.7},
    'SanAndres': {'latitude': 12.5, 'longitude': -81.7},
    'Santander': {'latitude': 7.6, 'longitude': -73.1},
    'Sucre': {'latitude': 9.0, 'longitude': -75.2},
    'Tolima': {'latitude': 4.0, 'longitude': -75.2},
    'ValleCauca': {'latitude': 3.4, 'longitude': -76.5},
    'vaupes': {'latitude': 0.7, 'longitude': -69.5},
    'Vichada': {'latitude': 4.0, 'longitude': -69.3}
}


In [84]:
# Supongamos que tu DataFrame se llama data
data['latitude'] = data['Departamento'].map(lambda x: departments_coordinates.get(x, {}).get('latitude'))
data['longitude'] = data['Departamento'].map(lambda x: departments_coordinates.get(x, {}).get('longitude'))


In [85]:
data.sample(10)

,time,P,G(i),H_sun,T2m,WS10m,Departamento,latitude,longitude
848580,2018-10-29 12:30:00,36.22,91.60,24.04,9.81,0.83,Caldas,5.1,-75.5
1642140,2019-05-17 12:30:00,134.99,299.46,21.88,18.29,0.90,ValleCauca,3.4,-76.5
499897,2021-01-15 01:30:00,0.00,0.00,0.00,17.79,0.41,Quindio,4.5,-75.6
1023270,2020-10-04 06:30:00,0.00,0.00,0.00,28.01,2.21,SanAndres,12.5,-81.7
578361,2023-12-29 09:30:00,0.00,0.00,0.00,9.91,0.48,Boyaca,5.5,-73.4
61216,2018-12-26 16:30:00,454.92,1087.84,57.91,23.11,2.07,NorteSantander,7.8,-72.9
1529868,2018-07-25 12:30:00,213.24,477.89,23.15,21.62,1.17,Guaviare,3.2,-72.6
210666,2018-01-14 18:30:00,372.08,876.53,58.13,21.43,1.03,Tolima,4.0,-75.2
489832,2019-11-22 16:30:00,287.37,658.75,64.84,20.10,0.55,Quindio,4.5,-75.6
1382347,2019-09-24 19:30:00,207.75,491.18,49.40,29.47,0.83,Sucre,9.0,-75.2


In [86]:
# VErify it all worked correctly
data.isna().sum()

time            0
P               0
G(i)            0
H_sun           0
T2m             0
WS10m           0
Departamento    0
latitude        0
longitude       0
dtype: int64

In [87]:
data['P'].std()

np.float64(137.6577413256713)

Now, we associate a score based on the characteristics

In [88]:
from sklearn.preprocessing import MinMaxScaler


In [89]:
from sklearn.preprocessing import MinMaxScaler

# Inicializamos el MinMaxScaler
scaler = MinMaxScaler()

# Seleccionamos las columnas a normalizar
columns_to_normalize = ['G(i)', 'H_sun', 'T2m', 'WS10m', 'P']

# Ajustamos el scaler y transformamos las columnas seleccionadas
scaled_data = scaler.fit_transform(data[columns_to_normalize])

# Convertir el resultado escalado de nuevo a un DataFrame con las mismas columnas
scaled_df = pd.DataFrame(scaled_data, columns=columns_to_normalize)

# Ahora scaled_df contiene las columnas escaladas, pero el DataFrame original no se modifica



In [ ]:
# Asignar los pesos
weights = {
    'G_i': 0.70,
    'h_sun': 0.10,
    'T2m': 0.10,
    'WS10m': 0.10,
}

# Calcular el score de viabilidad utilizando las columnas escaladas
data['viability_score'] = (scaled_df['G(i)'] * weights['G_i'] +
                          scaled_df['H_sun'] * weights['h_sun'] +
                          scaled_df['T2m'] * weights['T2m'] +
                          scaled_df['WS10m'] * weights['WS10m']) 


In [107]:
data.sample(10)

,time,P,G(i),H_sun,T2m,WS10m,Departamento,latitude,longitude,viability_score
725014,2022-09-22 22:30:00,141.10,329.28,6.06,28.88,0.28,Putumayo,1.0,-75.4,0.170794
430510,2019-02-14 22:30:00,102.75,242.44,6.60,28.90,0.90,Cesar,10.4,-73.8,0.151232
620441,2022-10-17 17:30:00,302.45,675.27,71.29,16.13,1.03,Cundinamarca,4.1,-74.2,0.297630
1234217,2020-10-29 17:30:00,335.86,781.17,68.57,21.96,0.90,Risaralda,5.0,-75.7,0.338527
617188,2022-06-04 04:30:00,0.00,0.00,0.00,10.47,0.62,Cundinamarca,4.1,-74.2,0.027646
132247,2021-02-02 07:30:00,0.00,0.00,0.00,25.88,1.52,Magdalena,9.9,-74.2,0.074608
587173,2018-12-31 13:30:00,180.02,382.46,31.59,10.47,1.52,Cundinamarca,4.1,-74.2,0.165731
432024,2019-04-19 00:30:00,0.00,0.00,0.00,24.67,1.17,Cesar,10.4,-73.8,0.069627
642218,2019-04-13 02:30:00,0.00,0.00,0.00,24.20,0.83,Caqueta,1.6,-75.6,0.066747
327331,2019-05-08 19:30:00,273.89,636.27,48.20,26.69,1.66,Meta,3.6,-73.3,0.295823


In [98]:
data['viability_score'].max()

np.float64(0.7740527828231355)

In [ ]:
# Filtrar las filas donde G(i) > 0 
df_filtered = df[df['G(i)'] > 0]

# Ahora, calculamos el score promedio por departamento
# Utilizamos la función `groupby` para agrupar por departamento y luego calculamos el promedio de la columna 'viability_score'
department_score = df_filtered.groupby('Departamento')['viability_score'].mean().reset_index()

# Ver los primeros resultados
department_score.head()


,Departamento,viability_score
0,ValleCauca,0.329475
